# A simple agent-based model

## An interactive ABM

### Use !pip install to install the various libraries used in the program. Press Shift+Enter while on the selected cell of code you want to run.

In [13]:
!pip install --upgrade pip
!pip install git+https://github.com/AB-CE/abce.git@icis --upgrade 
!pip install tornado==4.5.3 --upgrade
!pip install plotly
!pip install jupyterlab_plotly


  Cloning https://github.com/AB-CE/abce.git (to revision icis) to c:\users\dom\appdata\local\temp\pip-req-build-hp1fhpr6
Requirement not upgraded as not directly required: flexx>=0.4.1 in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (0.4.1)
Requirement not upgraded as not directly required: future in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (0.16.0)
Requirement not upgraded as not directly required: dataset==0.8 in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (0.8.0)
Requirement not upgraded as not directly required: numpy>=1.10.2p in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (1.13.3)
Requirement not upgraded as not directly required: pandas>=0.17.1 in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (0.20.3)
Requirement not upgraded as not directly required: bokeh==0.12.16 in c:\users\dom\anaconda3\lib\site-packages (from abce==0.9.5b0) (0.12.16)
Requirement not upgraded as not directly required: t

distributed 1.19.1 has requirement tornado>=4.5.1, but you'll have tornado 4.3 which is incompatible.


  Using cached https://files.pythonhosted.org/packages/0a/29/01057551db50f718fda2afa0e42abdfccca4f8b18fa6163c59588ae8e991/tornado-4.5.3-cp36-cp36m-win_amd64.whl
  Found existing installation: tornado 4.3
    Uninstalling tornado-4.3:
      Successfully uninstalled tornado-4.3


abceconomics 0.9.5b0 has requirement tornado==4.3, but you'll have tornado 4.5.3 which is incompatible.
abce 0.9.5b0 has requirement tornado==4.3, but you'll have tornado 4.5.3 which is incompatible.


abceconomics 0.9.5b0 has requirement tornado==4.3, but you'll have tornado 4.5.3 which is incompatible.
abce 0.9.5b0 has requirement tornado==4.3, but you'll have tornado 4.5.3 which is incompatible.


  Could not find a version that satisfies the requirement jupyterlab_plotly (from versions: )
No matching distribution found for jupyterlab_plotly


### The next cell contains all the parameters that are used to vary the model

In [ ]:
import abce
import pandas as pd

from firm import Firm
from people import People


params = dict(
    population=1000,
    people_money=1000,
    num_firms=20,
    firm_money=2000,

    num_days=2000,

    l=0.5,  # constant from CS equation

    num_days_buffer=10,  # number of days worth of wages a firm will keep after giving profits

    phi_upper=10,  # phi_upper * demand gives upper bound to inventory
    phi_lower=2,
    excess=1.1,  # if number of workers offered to work for firm exceeds 110% of ideal number, decrease wage
    wage_increment=0.01,
    price_increment=0.01,
    worker_increment=0.01,
    productivity=1,
    wage_acceptance=1)

## You can now run the simulation:

In [ ]:
simulation = abce.Simulation(name='economy', processes=1)
group_of_firms = simulation.build_agents(Firm, "firm", number=params["num_firms"], **params)
people = simulation.build_agents(People, "people", number=1, **params)

for r in range(params["num_days"]):
    simulation.time = r

    group_of_firms.panel_log(variables=['wage', 'ideal_num_workers'], goods=['workers'])
    people.create_labor()

    vacancies_list = list(group_of_firms.publish_vacencies())

    people.send_workers(vacancies_list)

    group_of_firms.production()
    group_of_firms.pay_workers()
    group_of_firms.pay_dividents()
    group_of_firms.send_prices()
    people.get_prices()
    demand = people.buy_goods()

    group_of_firms.sell_goods()
    group_of_firms.determine_bounds(demand=list(demand)[0])
    (group_of_firms + people).print_possessions()
    group_of_firms.determine_wage()
    group_of_firms.expand_or_change_price()
    (people + group_of_firms).destroy_unused_labor()
    people.consumption()
    group_of_firms.determine_profits()

print('done')
path = simulation.path
print(path)
simulation.finalize()
df = pd.read_csv(path + "/panel_firm.csv")

df.head()

##### The code will start by importing the firm and people agents from firm.py and people.py. 

At the top of the code there is a dictionary containing all the parameters, you can change these parameters to get different final effects on the outcome of the simulation.

Then, the code will loop over each day and perform various methods called from the firm and people agents. 

### Each day:

1) First the people agent will create labour for the day to be distributed between the firms.

2) The group of firms will then send the people agent the list of vacancies they are offering based on the various amounts of resources they have and value for certain parameters. (details can be found by looking at firm.py)

3) The people will then send the workers based on the vacancies

4) For each worker, one good is produced in a day

5) After production, the workers are paid, the profits after the firm's buffer has been stored are sent to the bosses (represented in the same "people" agent)

6) Then the bounds on inventory and prices are changed based on demand etc.

7) Wages are also calculated for the next round

8) Then the unused labour is destoyed and the people consume goods

9) Repeat for the next day...

The results are then stored in a pandas DataFrame.



### By running the next cell you can see the general layout of this dataset and some of the variables that you can graph.

In [ ]:
df.head()

### This next cell of code contains a new function that will let you produce a graph of one of the variables from the DataFrame.

In [ ]:
# Plotting method
import plotly.plotly as py
import plotly.graph_objs as go


def GraphFn(graphing_variable):
    """
    function that takes in graphing variable as parameter and the produces a graph
    using plotly
    """
    x_data = [[] for _ in range(params["num_firms"])]
    y_data = [[] for _ in range(params["num_firms"])]

    for i in range(len(df)):
        name = df["name"][i]
        number = int(name[4:])
        x_data[number].append(df["round"][i])
        y_data[number].append(df[graphing_variable][i])

    data = []    

    for i in range(params["num_firms"]):
        data.append(go.Scatter(x = x_data[i],
                            y = y_data[i],
                            mode = "lines"))
                            #name = ("firm" + str(i))))


    import plotly.offline as offline

    offline.init_notebook_mode(connected=True)
    offline.iplot(data)

### You can now look at the values for a variable from the firms, for example if you run the next cell you will be able to see the time evolution of each firms production:

In [ ]:
GraphFn("production")

## Have a play around calling "GraphFn()" for other variables or edit some of the parameters in the main body of the code